In [1]:
!git clone https://github.com/AshishJangra27/datasets


Cloning into 'datasets'...
remote: Enumerating objects: 344, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 344 (delta 23), reused 54 (delta 9), pack-reused 235 (from 1)
Receiving objects: 100% (344/344), 280.65 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (149/149), done.
Updating files: 100% (231/231), done.


In [2]:
import pandas as pd

df = pd.read_csv('/content/datasets/Myntra Products/products.csv')
df.head(2)

,Unnamed: 0,product_name,brand_name,rating,rating_count,marked_price,discounted_price,sizes,product_link,img_link,product_tag,brand_tag,discount_amount,discount_percent,tags,seller_address,meta_description,meta_keyword
0,0,Slim Fit Casual Shirt,Greenfibre,0.0,0,1699,1699,"38,39,40,42,44",shirts/greenfibre/greenfibre-men-yellow-slim-f...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",shirts,greenfibre,0,0,"clothing,men-clothing,shirts,greenfibre-shirts",Gujarat - 380006,Buy Greenfibre Men Yellow Slim Fit Checked Cas...,Greenfibre Men Yellow Slim Fit Checked Casual ...
1,1,Men Printed T-shirt,FREESOUL,0.0,0,799,799,"S,M,L,XL,XXL",tshirts/freesoul/freesoul-men-white-printed-co...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",tshirts,freesoul,0,0,"clothing,men-clothing,tshirts,freesoul-tshirts",Karnataka - 560058,Buy FREESOUL Men White Printed Cotton T Shirt ...,"FREESOUL Men White Printed Cotton T Shirt, FR..."


In [3]:
df['features'] = df['product_name'] + ' ' +  df['product_tag'] + ' ' + df['brand_tag']
df['tags'] = ',' + df['tags']
df['label'] = [' __label__'.join(tag.split(',')).strip().lower() for tag in df['tags']]

df.head(2)

,Unnamed: 0,product_name,brand_name,rating,rating_count,marked_price,discounted_price,sizes,product_link,img_link,product_tag,brand_tag,discount_amount,discount_percent,tags,seller_address,meta_description,meta_keyword,features,label
0,0,Slim Fit Casual Shirt,Greenfibre,0.0,0,1699,1699,"38,39,40,42,44",shirts/greenfibre/greenfibre-men-yellow-slim-f...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",shirts,greenfibre,0,0,",clothing,men-clothing,shirts,greenfibre-shirts",Gujarat - 380006,Buy Greenfibre Men Yellow Slim Fit Checked Cas...,Greenfibre Men Yellow Slim Fit Checked Casual ...,Slim Fit Casual Shirt shirts greenfibre,__label__clothing __label__men-clothing __labe...
1,1,Men Printed T-shirt,FREESOUL,0.0,0,799,799,"S,M,L,XL,XXL",tshirts/freesoul/freesoul-men-white-printed-co...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",tshirts,freesoul,0,0,",clothing,men-clothing,tshirts,freesoul-tshirts",Karnataka - 560058,Buy FREESOUL Men White Printed Cotton T Shirt ...,"FREESOUL Men White Printed Cotton T Shirt, FR...",Men Printed T-shirt tshirts freesoul,__label__clothing __label__men-clothing __labe...


In [4]:
data = [i[-1] + ' ' + i[-2] for i in df.values]

fd = open('data.txt','w')
for i in data:
  fd.write(i)
  fd.write('\n')

fd.close()


In [17]:
import fasttext

model = fasttext.train_supervised(input = 'data.txt')

In [25]:
import fasttext
import numpy as np

def _patched_predict(self, text, k=1, threshold=0.0, on_unicode_error="strict"):
    if isinstance(text, (list, tuple)):
        all_labels = []
        all_probs = []
        for t in text:
            labels, probs = self.predict(t, k=k, threshold=threshold, on_unicode_error=on_unicode_error)
            all_labels.append(labels)
            all_probs.append(probs)
        return all_labels, np.asarray(all_probs)

    # Ensure it's a proper string (fasttext C++ expects str, not bytes)
    if not isinstance(text, str):
        text = str(text)

    predictions = self.f.predict(text, k, threshold, on_unicode_error)

    if predictions is None:
        probs, labels = ([], ())
    else:
        labels, probs = zip(*predictions)
        labels = tuple(labels)
        probs = list(probs)

    # NumPy 2.0 fix
    return labels, np.asarray(probs)

# Apply the patch
fasttext.FastText._FastText.predict = _patched_predict


In [26]:
import numpy as np

for i in range(50):
  txt = df['features'][np.random.randint(len(df['features']))]
  print(txt)
  print(model.predict(txt.lower(), k = 5)[0])
  print('-'*30)

Unisex 200TC Printed Apron aprons soumya
(0.0003074939304497093, 0.0003042299358639866, 0.00028568936977535486, 0.00026279210578650236, 0.0002599371364340186)
------------------------------
Yoke Design Kurta with Palazzos & Dupatta kurta-sets khushal-k
(0.11172667890787125, 0.013204588554799557, 0.013123657554388046, 0.012755532748997211, 0.010693024843931198)
------------------------------
Face Wash Fine Gramflour face-wash-and-cleanser nivea
(0.002235864056274295, 0.0014450575690716505, 0.0013576509663835168, 0.0013289351481944323, 0.0012838587863370776)
------------------------------
Women Mules Flats flats ginger-by-lifestyle
(0.45193254947662354, 0.2095634937286377, 0.13673077523708344, 0.0699092373251915, 0.011020170524716377)
------------------------------
Women Solid Ribbed Pullover sweaters mango
(0.0023242735769599676, 0.0015659085474908352, 0.0007603427511639893, 0.0006517951842397451, 0.0005223277839832008)
------------------------------
Women Ethnic Motifs Printed Kurta ku